In [1]:
import os
import pandas as pd
import numpy as np

from lifd.lifd import LiFD, LIFD_COL, LIFD_SUP_COL, LIFD1_SUP_COL, LIFD2_SUP_COL
from lifd.databases.hotspots_database import HotspotsDB
from lifd.databases.cgi_database import CgiDB
from lifd.databases.oncokb_database import OncoKBDB
from lifd.databases.cosmic_db import CosmicDB
from lifd.predictors.vep import Vep
from lifd.predictors.candra import Candra
from lifd.predictors.cravat import Cravat
from lifd.predictors.cgi import Cgi
from lifd.predictors.fathmm import FatHMM
from lifd.settings import HOTSPOTS_FP, ONCOGENIC_VARS_FP, ONCOKB_ALLVARS_FP, COSMIC_VARS_FP, OUTPUT_DIR
from lifd.utils import FUNC_COL, init

INFO:lifd.lifd:VarCode is used for mutation effect prediction.


##### Either create a ```cgi_settings.py``` file with this content and import it or manually set these variables here:
```
CGI_USER_ID = '<YOUR_CGI_USERNAME>'
CGI_TOKEN = '<YOUR_CGI_TOKEN>'
```

In [2]:
from cgi_settings import CGI_USER_ID, CGI_TOKEN
# from lifd.excluded_module.cgi_settings import CGI_USER_ID, CGI_TOKEN
# CGI_USER_ID = '<YOUR_CGI_USERNAME>'
# CGI_TOKEN = '<YOUR_CGI_TOKEN>'
Cgi.set_login(CGI_USER_ID, CGI_TOKEN)

output_dir = init()

reference_genome = 'hg19'
suppl_table1_fp = 'SupplementaryTable3.xlsx'

het_cats = ['Truncal', 'Branched']

input_fp = suppl_table1_fp

from pathlib import Path
input_p = Path(input_fp)
output_fn = '{}_LiFDed.xlsx'.format(input_p.stem)
output_fp = os.path.join(input_p.parent, output_fn)

if input_p.suffix == '.xlsx' or input_p.suffix == '.xls':
    var_df = pd.read_excel(input_fp, comment='#')
    var_df[FUNC_COL] = True
    print('Read file with {} variants: {}'.format(len(var_df), input_fp))

INFO:lifd.predictors.cgi:Successfully set new username reiter.j@gmail.com and token.
INFO:lifd.utils:Output directory: /Users/reiter/workspaces/ped_gits/met_heterogeneity/analysis/lifd_examples/TMP


Read file with 103 variants: SupplementaryTable3.xlsx


In [3]:
# set up databases for first phase of LiFD
hs_db = HotspotsDB(HOTSPOTS_FP)
cgi_db = CgiDB(ONCOGENIC_VARS_FP)
oncokb_db = OncoKBDB(ONCOKB_ALLVARS_FP)
cosmic_db = CosmicDB(COSMIC_VARS_FP)
dbs = [hs_db, cgi_db, oncokb_db, cosmic_db
      ]

# set up predictors for second phase of LiFD
prds = [Vep, Candra, Cravat, FatHMM, Cgi]
# prds = [FatHMM]
lifd = LiFD(databases=dbs, predictors=prds, driver_gene_fp=None, 
            driver_gene_col='TCGADrClf', 
#             driver_gene_col='TokheimDrClf',
            reference_genome=reference_genome)

INFO:lifd.mutation_effects:Set reference genome to hg19.
INFO:lifd.lifd:Using databases: Hotspots, CGI_Catalog, OncoKB, COSMIC
INFO:lifd.lifd:Using predictors: Vep, Candra, Cravat, FatHMM, Cgi
INFO:lifd.lifd:No file with driver genes was given. Using column with driver gene classification: TCGADrClf
INFO:lifd.lifd:Initialized LiFD 0.1.0 with 4 databases and 5 predictors.


In [4]:
var_df = lifd.run_lifd(var_df, output_dir=output_dir, export_fn=output_fn)

INFO:lifd.lifd:LIFD output directory: /Users/reiter/workspaces/ped_gits/met_heterogeneity/analysis/lifd_examples/TMP
INFO:lifd.lifd:LiFD predicts that 44 of 82 (53.7%) nonsynonymous or splice-site variants in driver genes are functional.
INFO:lifd.lifd:Positive predictions of LiFD1 27/82 (32.9%)
INFO:lifd.lifd:Positive predictions of LiFD2 29/82 (35.4%)
INFO:lifd.lifd:Exported 103 annotated variants (44 LiFD) to TMP/SupplementaryTable3_LiFDed.xlsx.


In [5]:
print('Number of truncal and branched nonsynonymous or splice site mutations in TCGA driver genes: {} vs {}'.format(
    len(var_df[(var_df.Clonality == het_cats[0]) & var_df.TCGADrClf]), 
    len(var_df[(var_df.Clonality == het_cats[1]) & var_df.TCGADrClf])))
print('Number of truncal and branched LiFDs: {} vs {}'.format(
    len(var_df[(var_df.Clonality == het_cats[0]) & var_df.LiFD]), 
    len(var_df[(var_df.Clonality == het_cats[1]) & var_df.LiFD])))
# note that variants that did not reach present or absent probabilities of at least 80% across
# all samples were filtered out to minimize any biases from low sequencing depth or low neoplastic cell content

# count number of functional drivers per subject
# always use original dataframe to calculate means or medians because otherwise subjects with zero of 
# the considered mutations would be excluded from the summary statistic
# for example, np.mean(lifd_df[(lifd_df.Clonality == 'Clonal')].groupby(['Subject'])['LiFD'].agg('count')) 
# gives 2.4 instead of 2.1 clonal LiFDs
print('Mean number of truncal and branched LiFDs per subject: {:.2g} vs 0.0'.format(
    np.mean(var_df.groupby(['Subject', 'LiFD', 'Clonality']).size().unstack(level=[1,2], fill_value=0)[True, het_cats[0]]), 
#     np.mean(var_df.groupby(['Subject', 'LiFD', 'Clonality']).size().unstack(level=[1,2], fill_value=0)[True, het_cats[1]])
    ))

Number of truncal and branched nonsynonymous or splice site mutations in TCGA driver genes: 68 vs 14
Number of truncal and branched LiFDs: 44 vs 0
Mean number of truncal and branched LiFDs per subject: 2.6 vs 0.0


### Reanalyze data with a different driver gene list from Tokheim et al, PNAS 2016 (Table S1)

In [6]:
lifd_tu = LiFD(databases=dbs, predictors=prds, driver_gene_fp=None, driver_gene_col='TokheimDrClf')
var_tu_df = lifd_tu.run_lifd(var_df, output_dir=output_dir, export_fn=None)

INFO:lifd.mutation_effects:Set reference genome to hg19.
INFO:lifd.lifd:Using databases: Hotspots, CGI_Catalog, OncoKB, COSMIC
INFO:lifd.lifd:Using predictors: Vep, Candra, Cravat, FatHMM, Cgi
INFO:lifd.lifd:No file with driver genes was given. Using column with driver gene classification: TokheimDrClf
INFO:lifd.lifd:Initialized LiFD 0.1.0 with 4 databases and 5 predictors.
INFO:lifd.lifd:LIFD output directory: /Users/reiter/workspaces/ped_gits/met_heterogeneity/analysis/lifd_examples/TMP
INFO:lifd.lifd:LiFD predicts that 44 of 88 (50.0%) nonsynonymous or splice-site variants in driver genes are functional.
INFO:lifd.lifd:Positive predictions of LiFD1 27/88 (30.7%)
INFO:lifd.lifd:Positive predictions of LiFD2 29/88 (33.0%)


In [7]:
print('Number of clonal and subclonal nonsynonymous or splice site mutations in TCGA driver genes: {} vs {}'.format(
    len(var_tu_df[(var_tu_df.Clonality == het_cats[0]) & var_tu_df.TCGADrClf]), 
    len(var_tu_df[(var_tu_df.Clonality == het_cats[1]) & var_tu_df.TCGADrClf])))
print('Number of clonal and subclonal LiFDs: {} vs {}'.format(
    len(var_tu_df[(var_tu_df.Clonality == het_cats[0]) & var_tu_df.LiFD]), 
    len(var_tu_df[(var_tu_df.Clonality == het_cats[1]) & var_tu_df.LiFD])))

# count number of functional drivers per subject
print('Mean number of clonal and subclonal LiFDs per subject: {:.1f} vs {:.2f}'.format(
    np.mean(var_tu_df.groupby(['Subject', 'LiFD', 'Clonality']).size().unstack(level=[1,2], fill_value=0)[True, het_cats[0]]), 
    np.mean(var_tu_df.groupby(['Subject', 'LiFD', 'Clonality']).size().unstack(level=[1,2], fill_value=0)[True, het_cats[1]])))

Number of clonal and subclonal nonsynonymous or splice site mutations in TCGA driver genes: 68 vs 14
Number of clonal and subclonal LiFDs: 43 vs 1
Mean number of clonal and subclonal LiFDs per subject: 2.5 vs 0.06
